CS5010, Summer 2020 Capstone 1 Web Scraper
Data Understanding </H3>
    
Data Features:

___Initial Data Collection Report___: The Data Collection Report is a simple listing of the data sources acquired along with their locations, the procedures used to procure them and any difficulties encountered. This section will will aid both with future replication of this project and with the execution of similar future projects.</li>

___Data Description Report___: The Data Description report describes the data that has been procured including its format, quantity: for example, the count of records and fields in each table, the characteristics of the “fields,” and any other surface characteristics discovered. Here is where I have evaluated whether or not the data acquired satisfies the analysis requirements.

___Exploratory Data Analysis (EDA)___: EDA describes the results of exploring the data involved in this project. EDA includes the first findings and or initial hypothesis and their impact on the remainder of this project. I have also included graphs and plots to indicate data characteristics that suggest further examination and analysis of new data subsets.

___Data Quality Report___: The Data Quality report lists the results of the data quality verification along with solutions vetted by subject matter experts to data quality.
    
__Note:__

At this moment, the organization has no plans to procure external databases or invest in dispatching teams to each site as its engineers, analysts, and managers are busy managing the data they currently have. At some point, however, they might want to consider an extended deployment of data mining results, in which case purchasing additional IT infrastructure to capture sensor data not centrally registered may be quite useful. It may also be helpful to have demographic information to see how the maintainers across regions vary in skill set and maintenance practices.

__Initial Data Collection Report__

    The dataset was put together by the Grouplens research group at the University of Minnesota. It comprises 1, 10, and 20 million ratings. And can be found at https://grouplens.org/datasets/movielens/. The following Data Collection Report is a simple listing of the data sources acquired along with their locations, the procedures used to procure them, and any difficulties encountered. This section will aid both with future replication of this project and with the execution of similar future projects.

__Data Collection:__

1.	Data Source: MovieLens (
2.	Location: https://grouplens.org/datasets/movielens/
3.	Movie Ratings: Small: 100,000 ratings and 3,600 tag applications applied to 9,000 movies by 600 users. Last updated 9/2018
4.	Method: This dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100836 ratings and 3683 tag applications across 9742 movies. These data were created by 610 users between March 29, 1996 and September 24, 2018. This dataset was generated on September 26, 2018.
a.	Users were selected at random for inclusion. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided.
b.	The data are contained in the files links.csv, movies.csv, ratings.csv and tags.csv. More details about the contents and use of all these files follows.
c.	This and other GroupLens data sets are publicly available for download at http://grouplens.org/datasets/.
5.	Obstacles: This is a development dataset. As such, it may change over time and is not an appropriate dataset for shared research results. See available benchmark datasets if that is your intent.
    
    
__Movie Ids__
Only movies with at least one rating or tag are included in the dataset. These movie ids are consistent with those used on the MovieLens web site (e.g., id 1 corresponds to the URL https://movielens.org/movies/1). Movie ids are consistent between ratings.csv, tags.csv, movies.csv, and links.csv (i.e., the same id refers to the same movie across these four data files).

__Ratings Data File Structure (ratings.csv)__
All ratings are contained in the file ratings.csv. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

userId,movieId,rating,timestamp
The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.

In [19]:
# imports
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import numpy as np
import re
import string
from IPython.core.display import HTML

# settings
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# data viz imports
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# building scrapper: writing elegant parsing/scrapping function to parse data from internet
def get_soup(url):
    """Constructs and returns a soup using the HTML content of `url` passed"""
    # initialize a session
    session = requests.Session()
    # make the request
    html = session.get(url)
    # return the soup
    return bs(html.content, "html.parser")
 
def get_tables(soup):
    """Extracts and returns all tables in a soup object"""
    return soup.find_all("table",{"id":"usa_table_countries_today"})

def get_table_headers(table):
    """Given a table soup, returns all the headers"""
    headers = []
    for th in table.find("tr").find_all("th"):
        headers.append(th.text.strip())
    return headers
 
def get_table_rows(table):
    """Given a table, returns all its rows"""
    rows = []
    for tr in table.find_all("tr")[1:]:
        cells = []
        # grab all td tags in this table row
        tds = tr.find_all("td")
        if len(tds) == 0:
            # if no td tags, search for th tags
 
            ths = tr.find_all("th")
            for th in ths:
                cells.append(th.text.strip())
        else:
            # use regular td tags
            for td in tds:
                cells.append(td.text.strip())
        rows.append(cells)
    return rows

def main(url):
    # get the soup
    soup = get_soup(url)
    # extract all the tables from the web page
    tables = get_tables(soup)
    # iterate over all tables
    for i, table in enumerate(tables, start=1):
        # get the table headers
        headers = get_table_headers(table)
        # get all the rows of the table
        rows = get_table_rows(table)
        # save table as csv file
        table_name = f"table-{i}"
        save_as_csv(table_name, headers, rows)    

def save_as_csv(table_name, headers, rows):
    pd.DataFrame(rows, columns=headers).to_csv("data\covid19_states.csv", encoding='utf-8', index=False)

In [27]:
# get data source: df1_covidcases

# step1. instantiating scrapper function        
main("https://www.worldometers.info/coronavirus/country/us/")
# step2. setting up column names: col_list
col_list = ["USAState","TotalCases","NewCases","TotalDeaths","NewDeaths","ActiveCases"]
# step3. reading data in using read_csv, and casting col_list to headers: df1_covidcases
df1_covidcases = pd.read_csv("data/covid19_states.csv", usecols=col_list)


# get data source: df2_humidity

# step1. assign webpage address of interest that we will scrape
url = 'http://www.usa.com/rank/us--average-humidity--state-rank.htm#:~:text=Rank%20Average%20Humidity%20%E2%96%BC%20State%20%2F%20Population%201.,4.%2080.76%25%20Maine%20%2F%201%2C328%2C535%2047%20more%20rows'
# step2. use pandas read_html() tool to read and parse our site. The [0] indicates that we want to grab the first table on the webpage
df2_humidity = pd.read_html(url)[0]
# step3. set first row to headers
df2_humidity = df2_humidity.rename(columns=df2_humidity.iloc[0])
# step 4. drop duplicated header row inplace
df2_humidity.drop(df2_humidity.index[0], inplace=True)
# step 5. split data by the delimeter "/" into: 'State' and 'Population'
df2_humidity[['State','Population']] = df2_humidity['State / Population'].str.split("/",expand=True)
# step 6. drop 'State / Population' column
df2_humidity.drop(columns=['State / Population'], inplace=True)
# step 7. rename columns using dictionary
df2_humidity.rename(columns={'Average Humidity ▼' : 'Average Humidity', 'State' : 'USAState'},inplace=True)
# step 8. write file to .csv
df2_humidity.to_csv("data\covid19_states_humidity.csv", encoding='utf-8', index=False)

In [16]:
# check data frames from scrapper: df1_covidcases & df2_humidity
print(df1_covidcases.head())
print(df2_humidity.head())

        USAState TotalCases NewCases TotalDeaths  NewDeaths ActiveCases
0      USA Total  3,359,838   +4,192     137,436       33.0   1,731,677
1       New York    426,807      NaN      32,393        NaN     227,391
2     California    319,985      NaN       7,030        4.0     227,398
3          Texas    259,465      NaN       3,228        NaN     128,357
4        Florida    254,511      NaN       4,197        NaN     218,244
5     New Jersey    180,672      NaN      15,603        NaN      88,899
6       Illinois    154,094      NaN       7,369        NaN      31,278
7        Arizona    119,930      NaN       2,151        NaN     103,385
8        Georgia    114,401      NaN       2,996        NaN      93,715
9  Massachusetts    111,398      NaN       8,310        NaN       8,741
   Rank Average Humidity        USAState   Population
1    1.           82.01%           Iowa     3,078,116
2    2.           81.86%  New Hampshire     1,321,069
3    3.           81.46%         Alaska       

In [17]:
# check the info for df1_covidcases
# We see that there are some null values that we will need to work out. Aslo we may need to moralize values within each record for leading and lagging spaces.
print(df1_covidcases.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   USAState     64 non-null     object 
 1   TotalCases   64 non-null     object 
 2   NewCases     15 non-null     object 
 3   TotalDeaths  62 non-null     object 
 4   NewDeaths    9 non-null      float64
 5   ActiveCases  64 non-null     object 
dtypes: float64(1), object(5)
memory usage: 3.1+ KB
None


In [18]:
# check the info for df1_covidcases
# We see that the values listed here are a lot more pure. To merge the dataframes we just need to esure primary and foreign keys are consistent on USAState
print(df2_humidity.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51 entries, 1 to 51
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Rank              51 non-null     object
 1   Average Humidity  51 non-null     object
 2   USAState          51 non-null     object
 3   Population        51 non-null     object
dtypes: object(4)
memory usage: 2.0+ KB
None


In [22]:
# merge wrangled dataframes
# we see that there is an issue with the primary and foreign keys i.e. lagging space or even spelling
test_merge = pd.merge(df1_covidcases,df2_humidity, how='right', on='USAState')
test_merge.head()

,USAState,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,Rank,Average Humidity,Population
0,Iowa,NaN,NaN,NaN,NaN,NaN,1.,82.01%,"3,078,116"
1,New Hampshire,NaN,NaN,NaN,NaN,NaN,2.,81.86%,"1,321,069"
2,Alaska,NaN,NaN,NaN,NaN,NaN,3.,81.46%,"728,300"
3,Maine,NaN,NaN,NaN,NaN,NaN,4.,80.76%,"1,328,535"
4,North Dakota,NaN,NaN,NaN,NaN,NaN,5.,80.74%,"704,925"


In [26]:
# here we look at value fields for primary keys to see why they are off. We will need to address this in the data preparation.
frames = [df1_covidcases['USAState'], df2_humidity['USAState']]
result = pd.concat(frames).sort_values()
result

19                         Alabama
36                        Alabama 
49                          Alaska
3                          Alaska 
7                          Arizona
14                        Arizona 
30                        Arkansas
31                       Arkansas 
2                       California
9                      California 
24                        Colorado
10                       Colorado 
21                     Connecticut
16                    Connecticut 
38                        Delaware
50                       Delaware 
62           Diamond Princess Ship
40            District Of Columbia
24           District of Columbia 
58                 Federal Prisons
4                          Florida
28                        Florida 
8                          Georgia
39                        Georgia 
60             Grand Princess Ship
52                            Guam
51                          Hawaii
47                         Hawaii 
41                  

In [28]:
# write data to .csv
# write out covid19_states.csv
df1_covidcases.to_csv("data\covid19_states_cases.csv", encoding='utf-8', index=False)

# write out covid19_states_humidity.csv
df2_humidity.to_csv("data\covid19_states_humidity.csv", encoding='utf-8', index=False)